In [7]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load raw dataset from fixed path(kaggle Link -https://www.kaggle.com/code/ahmdayman/retail-sales-dataset)
raw_data_path = r"C:\Users\SKY\Desktop\JOB prep\Project-1-Retail-Sales-Analysis\Data\retail_sales_dataset.csv"
df = pd.read_csv(raw_data_path)

print("Raw Dataset Shape:", df.shape)
print("Raw Dataset Columns:", list(df.columns))

Raw Dataset Shape: (1000, 9)
Raw Dataset Columns: ['Transaction ID', 'Date', 'Customer ID', 'Gender', 'Age', 'Product Category', 'Quantity', 'Price per Unit', 'Total Amount']


In [8]:
# Data Quality Assessment
print("=== DATA QUALITY ASSESSMENT ===")
print(f"Total Rows: {len(df)}")
print(f"Total Columns: {len(df.columns)}")
print("\nMissing Values:")
print(df.isnull().sum())
print(f"\nDuplicate Rows: {df.duplicated().sum()}")
print(f"Unique Transaction IDs: {df['Transaction ID'].nunique()}")

=== DATA QUALITY ASSESSMENT ===
Total Rows: 1000
Total Columns: 9

Missing Values:
Transaction ID      0
Date                0
Customer ID         0
Gender              0
Age                 0
Product Category    0
Quantity            0
Price per Unit      0
Total Amount        0
dtype: int64

Duplicate Rows: 0
Unique Transaction IDs: 1000


In [9]:
# Data Type Conversions and Cleaning
df_clean = df.copy()

# 1. Standardize column names (replace spaces with underscores, lowercase)
df_clean.columns = [col.lower().replace(' ', '_') for col in df_clean.columns]
print("New Column Names:", list(df_clean.columns))

# 2. Convert date column to datetime
df_clean['date'] = pd.to_datetime(df_clean['date'])
print(f"Date Range: {df_clean['date'].min()} to {df_clean['date'].max()}")

# 3. Standardize categorical data
df_clean['gender'] = df_clean['gender'].str.upper()
df_clean['product_category'] = df_clean['product_category'].str.upper()

print("\nProduct Categories:", df_clean['product_category'].unique())
print("Gender Values:", df_clean['gender'].unique())


New Column Names: ['transaction_id', 'date', 'customer_id', 'gender', 'age', 'product_category', 'quantity', 'price_per_unit', 'total_amount']
Date Range: 2023-01-01 00:00:00 to 2024-01-01 00:00:00

Product Categories: ['BEAUTY' 'CLOTHING' 'ELECTRONICS']
Gender Values: ['MALE' 'FEMALE']


In [10]:
# 4. Create derived fields for analysis
df_clean['year'] = df_clean['date'].dt.year
df_clean['month'] = df_clean['date'].dt.month
df_clean['quarter'] = df_clean['date'].dt.quarter
df_clean['day_of_week'] = df_clean['date'].dt.day_name()
df_clean['month_name'] = df_clean['date'].dt.month_name()
df_clean['is_weekend'] = df_clean['date'].dt.weekday >= 5

# Revenue and profit calculations (assuming cost structure)
df_clean['revenue'] = df_clean['total_amount']  # Total amount is revenue
df_clean['unit_cost'] = df_clean['price_per_unit'] * 0.6  # Assume 40% margin
df_clean['total_cost'] = df_clean['unit_cost'] * df_clean['quantity']
df_clean['profit'] = df_clean['revenue'] - df_clean['total_cost']
df_clean['profit_margin'] = df_clean['profit'] / df_clean['revenue']

# Age group segmentation
bins = [0, 25, 35, 45, 55, 100]
labels = ['18-25', '26-35', '36-45', '46-55', '56+']
df_clean['age_group'] = pd.cut(df_clean['age'], bins=bins, labels=labels, right=False)

print(f"\nDerived fields added. New shape: {df_clean.shape}")


Derived fields added. New shape: (1000, 21)


In [11]:
# 5. Data validation checks
print("=== DATA VALIDATION ===")

# Check total amount calculation
calculated_total = df_clean['quantity'] * df_clean['price_per_unit']
validation_check = (df_clean['total_amount'] == calculated_total).all()
print(f"Total Amount validation: {'PASSED' if validation_check else 'FAILED'}")

# Check for negative values
negative_checks = {
    'quantity': (df_clean['quantity'] < 0).sum(),
    'price_per_unit': (df_clean['price_per_unit'] < 0).sum(), 
    'total_amount': (df_clean['total_amount'] < 0).sum(),
    'profit': (df_clean['profit'] < 0).sum()
}
print("Negative value checks:", negative_checks)

# Age validation
age_issues = df_clean[(df_clean['age'] < 18) | (df_clean['age'] > 100)]
print(f"Age outliers (< 18 or > 100): {len(age_issues)}")

print("\nFinal cleaned dataset shape:", df_clean.shape)

=== DATA VALIDATION ===
Total Amount validation: PASSED
Negative value checks: {'quantity': 0, 'price_per_unit': 0, 'total_amount': 0, 'profit': 0}
Age outliers (< 18 or > 100): 0

Final cleaned dataset shape: (1000, 21)


In [12]:
# 6. Save cleaned dataset
cleaned_data_path = r"C:\Users\SKY\Desktop\JOB prep\Project-1-Retail-Sales-Analysis\Data\retail_sales_cleaned.csv"
df_clean.to_csv(cleaned_data_path, index=False)
print(f"Cleaned dataset saved to: {cleaned_data_path}")

# Display sample of cleaned data
print("\nSample of cleaned data:")
print(df_clean.head())

Cleaned dataset saved to: C:\Users\SKY\Desktop\JOB prep\Project-1-Retail-Sales-Analysis\Data\retail_sales_cleaned.csv

Sample of cleaned data:
   transaction_id       date customer_id  gender  age product_category  \
0               1 2023-11-24     CUST001    MALE   34           BEAUTY   
1               2 2023-02-27     CUST002  FEMALE   26         CLOTHING   
2               3 2023-01-13     CUST003    MALE   50      ELECTRONICS   
3               4 2023-05-21     CUST004    MALE   37         CLOTHING   
4               5 2023-05-06     CUST005    MALE   30           BEAUTY   

   quantity  price_per_unit  total_amount  year  ...  quarter  day_of_week  \
0         3              50           150  2023  ...        4       Friday   
1         2             500          1000  2023  ...        1       Monday   
2         1              30            30  2023  ...        1       Friday   
3         1             500           500  2023  ...        2       Sunday   
4         2           